In [ ]:
# Importing the necessary packages
import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
# Scraping top 1 agriculture pulications("Progressive Farmer")

# Search keyword: soybean, tariffs
url1 = requests.get("http://www.agrinews-pubs.com/search/?f=html&q=soybean+tariffs&d1=2017-09-01&d2=2019-10-14&s=start_time&sd=desc&l=100&t=article&nsa=eedition")
url2 = requests.get("http://www.agrinews-pubs.com/search/?f=html&q=soybean+tariffs&d1=2017-09-01&d2=2019-10-14&s=start_time&sd=desc&l=100&t=article&nsa=eedition&app%5B0%5D=editorial&o=100")

soup1 = BeautifulSoup(url1.content, 'html.parser')
soup2 = BeautifulSoup(url2.content, 'html.parser')

In [ ]:
# Get article link & title
content1 = soup1.find_all('h3', class_ = 'tnt-headline')
content2 = soup2.find_all('h3', class_ = 'tnt-headline')

# Get article date
date1 = soup1.find_all('time', class_ = "tnt-date asset-date text-muted")
date2 = soup2.find_all('time', class_ = "tnt-date asset-date text-muted")

# Empty lists for links, titles, dates and contents
list_links = []
list_titles = []
list_dates = []
list_contents = []

for n in np.arange(0, len(content1)):
    
    # Getting the link of the article
    link = 'http://www.agrinews-pubs.com' + content1[n].find('a')['href']
    list_links.append(link)
    
    # Getting the title, delete blanks
    title = content1[n].get_text().replace('\n\n        \n\n        \n            ', '')
    list_titles.append(title)
    
    # Getting the date
    date = date1[n].get_text()
    list_dates.append(date)
    #datetime.strptime(date1[0].get_text(), '%b %d, %Y')
    
    # Reading the content (it is divided in paragraphs)
    article = requests.get(link)
    soup_article = BeautifulSoup(article.content, 'html.parser')
    body = soup_article.find_all('p')
    
    # Unifying the paragraphs, delete unnecessary lines
    list_paragraphs = []
    for paragraph in body[3:-10]:
        paragraphtext = paragraph.get_text()
        list_paragraphs.append(paragraphtext)
        final_article = " ".join(list_paragraphs)
        
    list_contents.append(final_article)
    
for n in np.arange(0, len(content2)):
    
    link = 'http://www.agrinews-pubs.com' + content1[n].find('a')['href']
    list_links.append(link)
    
    title = content1[n].get_text().replace('\n\n        \n\n        \n            ', '')
    list_titles.append(title)
    
    date = date1[n].get_text()
    list_dates.append(date)
    
    article = requests.get(link)
    soup_article = BeautifulSoup(article.content, 'html.parser')
    body = soup_article.find_all('p')
    
    list_paragraphs = []
    for paragraph in body[3:-10]:
        paragraphtext = paragraph.get_text()
        list_paragraphs.append(paragraphtext)
        final_article = " ".join(list_paragraphs)
        
    list_contents.append(final_article)

In [ ]:
# save article data to file
data = {'Title':list_titles,
        'PageLink':list_links, 
        'Article':list_contents, 
        'Date':list_dates}

# oldnews = pd.read_excel('quartz\\news.xlsx')
news = pd.DataFrame(data = data)
cols = ['Title', 'PageLink', 'Article', 'Date']
news = news[cols]

news.to_csv("news.csv")